In [82]:
# !pip install langchain
# !pip install langchain_openai
# !pip install -U langchain-community

In [83]:
######################################################
# This script converts the dataset into a JSON file. #
######################################################

import pandas as pd
from ast import literal_eval
import os
import json
# Load the dataset
train_csv_path = '../../contest_baseline_code/data/raw/train.csv'
dataset = pd.read_csv(train_csv_path)

# Convert the dataset into a list of records
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'given_prompt': {            
            '지문': row['paragraph'],
            '질문': problems['question'],
            '선택지': problems['choices'],
            '정답': problems.get('answer', None),
        }
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)

# Save the records as a JSON file
train_json_path = '../../contest_baseline_code/data/raw/train.json'

with open(train_json_path, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

In [ ]:
import warnings
import os

warnings.filterwarnings('ignore') # warning을 보고 싶지 않을 경우 활성화

os.environ["OPENAI_API_KEY"] = "" # OpenAI API Key 입력

In [85]:
data_version = "gpt_4o_mini-1_0_2"
id_version = "gpt-4o-mini-1.0.2"

In [ ]:
######################################################
# This script augments the dataset using GPT-4o-mini #
######################################################

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks import get_openai_callback
from tqdm import tqdm
import os
import json

# Define language model
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.6,
)

# Define the output parser
parser = JsonOutputParser()

# Define the prompt template
template = "당신은 지금부터 한국의 수능 문제를 만들어야 합니다.\n#주어진 수능 문제#에서 영감을 얻어 #새로운 수능 문제#를 만드는 것이 목표입니다.\n이 #새로운 수능 문제#는 #주어진 수능 문제#와 같은 과목에 속해야 하지만 훨씬 더 희귀해야 합니다.\n#새로운 수능 문제#의 길이와 난이도는 #주어진 수능 문제#와 비슷해야 합니다.\n#새로운 수능 문제#는 합리적이어야 하며 사람이 이해하고 응답할 수 있어야 합니다.\n'#주어진 수능 문제#', '#새로운 수능 문제#', '주어진 수능 문제' 및 '새로운 수능 문제'는 #새로운 프롬프트#에 나타나서는 안됩니다.\n\n#주어진 수능 문제#:\n{given_prompt}\n\n#새로운 수능 문제#는 반드시 JSON 객체로 반환하세요.\n#새로운 수능 문제#:\n"

# Define the source and destination paths
SOURCE_PATH = '../../contest_baseline_code/data/raw/train.json'
DEST_PATH = f'../../contest_baseline_code/data/augmented/train_augmented_by_{data_version}.json'

# Load the source data
with open(SOURCE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

new_data = []
total_tokens = 0
total_prompt_tokens = 0
total_completion_tokens = 0
total_cost = 0.0

# # expriement: check the first 10 data
# data = data[:5]

for item in tqdm(data, desc="Augmenting"):
    # 3달러 이상 사용되면 작업 중단
    if total_cost > 3:
        print("총 비용 $3에 도달하면 작업 중단")
        break
    
    # Invoke the chain
    data_id = item["id"]
    given_prompt = str(item["given_prompt"])
    prompt = PromptTemplate(
        template=template, 
        input_variables=["given_prompt"]
        )

    # define the chain
    chain = prompt | llm | parser

    # invoke the chain, and get the callback -> cost is calculated here
    with get_openai_callback() as cb:
        try:
            response = chain.invoke(given_prompt)
        except Exception as e:
            print(f"Error while generating response for ID {data_id}: {e}")
            continue
        total_tokens += cb.total_tokens
        total_prompt_tokens += cb.prompt_tokens
        total_completion_tokens += cb.completion_tokens
        total_cost += cb.total_cost
    new_data.append((data_id,response))

# format the data
id_formatted_data = []
for data_id, response in new_data:
    id_formatted_data.append({
        "version": id_version,
        "id": f"{data_id}",
        "given_prompt": response
    })

# filter response "given_prompt" "answer" is 1,2,3,4,5
answer_filtered_data = [
    item for item in id_formatted_data 
    if 'given_prompt' in item and '정답' in item['given_prompt'] and item['given_prompt']['정답'] in [1, 2, 3, 4, 5]
]

# Save the augmented data    
with open(DEST_PATH, "w", encoding="utf-8") as f:
    json.dump(answer_filtered_data, f, ensure_ascii=False, indent=4)
    
print(f"총 사용된 토큰수: \t\t{total_tokens}")
print(f"프롬프트에 사용된 토큰수: \t{total_prompt_tokens}")
print(f"답변에 사용된 토큰수: \t{total_completion_tokens}")
print(f"호출에 청구된 금액(USD): \t${total_cost}")

In [ ]:
######################################################
# This script augments the dataset using GPT-4o-mini #
######################################################

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks import get_openai_callback
from tqdm import tqdm
import os
import json

# Define language model
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.6,
)

# Define the output parser
parser = JsonOutputParser()

# Define the prompt template
template = "당신은 지금부터 한국의 수능 문제를 만들어야 합니다.\n#주어진 수능 문제#에서 영감을 얻어 #새로운 수능 문제#를 만드는 것이 목표입니다.\n이 #새로운 수능 문제#는 #주어진 수능 문제#와 같은 과목에 속해야 하지만 훨씬 더 희귀해야 합니다.\n#새로운 수능 문제#의 길이와 난이도는 #주어진 수능 문제#와 비슷해야 합니다.\n#새로운 수능 문제#는 합리적이어야 하며 사람이 이해하고 응답할 수 있어야 합니다.\n'#주어진 수능 문제#', '#새로운 수능 문제#', '주어진 수능 문제' 및 '새로운 수능 문제'는 #새로운 프롬프트#에 나타나서는 안됩니다.\n\n#주어진 수능 문제#:\n{given_prompt}\n\n#새로운 수능 문제#는 반드시 JSON 객체로 반환하세요.\n#새로운 수능 문제#:\n"

# Define the source and destination paths
SOURCE_PATH = '../../contest_baseline_code/data/raw/train.json'
DEST_PATH = f'../../contest_baseline_code/data/augmented/train_augmented_by_{data_version}.json'

# Load the source data
with open(SOURCE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

new_data = []
total_tokens = 0
total_prompt_tokens = 0
total_completion_tokens = 0
total_cost = 0.0

# # expriement: check the first 10 data
# data = data[:5]

for item in tqdm(data, desc="Augmenting"):
    # 3달러 이상 사용되면 작업 중단
    if total_cost > 3:
        print("총 비용 $3에 도달하면 작업 중단")
        break
    
    # Invoke the chain
    data_id = item["id"]
    given_prompt = str(item["given_prompt"])
    prompt = PromptTemplate(
        template=template, 
        input_variables=["given_prompt"]
        )

    # define the chain
    chain = prompt | llm | parser

    # invoke the chain, and get the callback -> cost is calculated here
    with get_openai_callback() as cb:
        try:
            response = chain.invoke(given_prompt)
        except Exception as e:
            print(f"Error while generating response for ID {data_id}: {e}")
            continue
        total_tokens += cb.total_tokens
        total_prompt_tokens += cb.prompt_tokens
        total_completion_tokens += cb.completion_tokens
        total_cost += cb.total_cost
    new_data.append((data_id,response))

# format the data
id_formatted_data = []
for data_id, response in new_data:
    id_formatted_data.append({
        "version": id_version,
        "id": f"{data_id}",
        "given_prompt": response
    })

# filter response "given_prompt" "answer" is 1,2,3,4,5
answer_filtered_data = [
    item for item in id_formatted_data 
    if 'given_prompt' in item and '정답' in item['given_prompt'] and item['given_prompt']['정답'] in [1, 2, 3, 4, 5]
]

# Save the augmented data    
with open(DEST_PATH, "w", encoding="utf-8") as f:
    json.dump(answer_filtered_data, f, ensure_ascii=False, indent=4)
    
print(f"총 사용된 토큰수: \t\t{total_tokens}")
print(f"프롬프트에 사용된 토큰수: \t{total_prompt_tokens}")
print(f"답변에 사용된 토큰수: \t{total_completion_tokens}")
print(f"호출에 청구된 금액(USD): \t${total_cost}")

In [99]:
###############################################################
# This script converts the augmented dataset into a CSV file. #
###############################################################

import pandas as pd
import json

# json file path
json_path = f'../../contest_baseline_code/data/augmented/train_augmented_by_{data_version}.json'
csv_output_path = f'../../contest_baseline_code/data/augmented/train_augmented_by_{data_version}.csv'

# Load the json file
with open(json_path, 'r', encoding='utf-8') as f:
    records = json.load(f)

# Convert the records into a DataFrame
rows = []
for record in records:
    problems = {
        'question': record['given_prompt'].get('질문', None),
        'choices': record['given_prompt'].get('선택지', None),
        'answer': record['given_prompt'].get('정답', None),
    }
    
    row = {
        'id': f"{record.get('id', '')}_{record.get('version', '')}",
        'paragraph': record['given_prompt'].get('지문', None),
        'problems': str(problems)
    }        
    rows.append(row)

# Save the DataFrame as a CSV file
df = pd.DataFrame(rows)
df['question_plus'] = ""
print(df.shape)
df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')

(1248, 4)


In [ ]:
########################################################################
# This script filters the augmented dataset to remove invalid answers. #
########################################################################
import pandas as pd
from ast import literal_eval

augmented_data = pd.read_csv(csv_output_path)

print(augmented_data.shape)
augmented_data

(1248, 4)


,id,paragraph,problems,question_plus
0,generation-for-nlp-425_gpt-4o-mini-1.0.2,"한 관료가 상소하여 아뢰기를, “신이 고려시대의 송시열이 남긴 글을 보았는데, 유교...",{'question': '상소한 인물이 속한 시대에 대한 설명으로 옳은 것만을 모두...,NaN
1,generation-for-nlp-427_gpt-4o-mini-1.0.2,조선시대의 한 학자는 '과거의 역사와 문화가 현재와 미래에 미치는 영향은 지대하다'...,"{'question': '이 학자가 주장한 내용에 대한 설명으로 옳은 것은?', '...",NaN
2,generation-for-nlp-428_gpt-4o-mini-1.0.2,"고려시대에는 문신과 무신의 대립이 심화되었고, 이로 인해 정치적 갈등이 발생하였다....","{'question': '위 지문에서 ‘무신 정권’에 대한 설명으로 옳은 것은?',...",NaN
3,generation-for-nlp-429_gpt-4o-mini-1.0.2,"조선 후기, 상업의 발달과 함께 상인 계층이 형성되었다. 이들은 (가) 상업적 이익...","{'question': '(가) 상인들이 추구한 주된 이익은 무엇인가?', 'cho...",NaN
4,generation-for-nlp-431_gpt-4o-mini-1.0.2,"(가) 고려는 송나라와의 외교 관계를 통해 문화적 교류를 활발히 하였으며, 특히 불...","{'question': '(가) 고려 시대의 문화적 특징으로 옳은 것은?', 'ch...",NaN
...,...,...,...,...
1243,generation-for-nlp-2884_gpt-4o-mini-1.0.2,한국의 전자회사 삼성전자는 최근 인도네시아 자회사 '삼성전자 인도네시아'가 자국 내...,{'question': '삼성전자 인도네시아가 체결한 스마트폰 생산 공장의 계약금액...,NaN
1244,generation-for-nlp-2892_gpt-4o-mini-1.0.2,최근 한국의 여러 대학교가 온라인 강의를 확대하고 있다. 이는 학생들의 학습 편의성...,"{'question': '온라인 강의를 도입하기로 결정한 대학교는 어디인가?', '...",NaN
1245,generation-for-nlp-2894_gpt-4o-mini-1.0.2,서울시가 주최하는 '서울 청년 문화축제'가 오는 9월 15일부터 17일까지 개최된다...,"{'question': '서울 청년 문화축제의 주요 목적은 무엇인가?', 'choi...",NaN
1246,generation-for-nlp-2896_gpt-4o-mini-1.0.2,최근 한국의 IT 산업이 급격히 성장하면서 많은 스타트업들이 주목받고 있다. 특히 ...,{'question': '최근 매출이 150% 증가한 스타트업의 주요 분야는 무엇인...,NaN


In [101]:
raw_train_data = pd.read_csv(train_csv_path)
print(raw_train_data.shape)
raw_train_data

(2030, 4)


,id,paragraph,problems,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,NaN
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...",NaN
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...",NaN
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...",NaN
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,"{'question': '(가) 국가에 대한 설명으로 옳은 것은?', 'choice...",NaN
...,...,...,...,...
2025,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...","{'question': '잭 맥도널드가 남긴 유산의 총액은 얼마인가?', 'choi...",NaN
2026,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...","{'question': ""넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇...",NaN
2027,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,{'question': '옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇...,NaN
2028,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,{'question': '방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?'...,NaN
